## Download Messenger data
1. Go to https://www.facebook.com/dyi/?referrer=yfi_settings
1. Select last 1 year, media quality low, format JSON
1. "Deselect all", then select "Messages" and "Comments and reactions"
1. Set a 15m reminder to check for data with Vitamin R
1. Download data!
1. Click to unzip it

In [2]:
!rm -rf ~/code/crm/facebook-jasoncbenn
!mv ~/Downloads/facebook-jasoncbenn ~/code/crm/facebook-jasoncbenn

## Download Twitter data

In [1]:
from twitter_scraper import get_tweets

for tweet in get_tweets('from:@zan2434', pages=1):
    print(tweet['text'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Parse Messenger inbox

In [3]:
from datetime import datetime
import glob
import json

In [4]:
def parse_datetime(dt: int) -> datetime:
    return datetime.fromtimestamp(dt)

def parse_datetime_ms(dt: int) -> datetime:
    return datetime.fromtimestamp(dt / 1e3)

In [5]:
chats = glob.glob("facebook-jasoncbenn/messages/inbox/*/*.json")

In [6]:
from datetime import timedelta
import unidecode
from collections import defaultdict

MY_NAME = 'Jason Benn'

interactions = defaultdict(set)

for chat_json_path in chats:
    with open(chat_json_path, encoding='raw_unicode_escape') as f:
        chat = json.loads(f.read().encode('raw_unicode_escape').decode())

    others = [x['name'] for x in chat['participants'] if x['name'] != MY_NAME]

    my_messages = [x for x in chat['messages'] if x['sender_name'] == MY_NAME]

    if not len(my_messages):
        continue 

    last_message_time = None

    is_groupchat = len(others) > 1

    if is_groupchat:
        # Creative technique: anytime I message the group, I get "credit" for all messages within 2 days
        # Goal is to account for groups where I'm active + others are active - those feel like touches
        # But many other groups are too big, too sparse, not mine, etc. Those don't count.
        for message in reversed(chat['messages']):
            msg_time = parse_datetime_ms(message['timestamp_ms'])
            msg_name = unidecode.unidecode(message['sender_name'])
            if msg_name == MY_NAME:
                last_message_time = msg_time
                continue

            if last_message_time and last_message_time + timedelta(days=2) > msg_time:
                interactions[msg_name].add(msg_time.date())

    else:
        # If this is 1:1, just count every unique day of any message.
        for message in chat['messages']:
            msg_time = parse_datetime_ms(message['timestamp_ms'])
            msg_name = unidecode.unidecode(message['sender_name'])
            if msg_name == MY_NAME:
                continue
            interactions[msg_name].add(msg_time.date())

## Parse FB comments on other posts

In [7]:
import re

In [8]:
with open("facebook-jasoncbenn/comments_and_reactions/comments.json", encoding='raw_unicode_escape') as f:
    comments = json.loads(f.read().encode('raw_unicode_escape').decode())

In [13]:
for comment in comments['comments_v2']:
    comment_time = parse_datetime(comment['timestamp'])

    name = None
    
    if match := re.match("Jason Benn (replied to|commented on) his own (photo|post|comment|video|GIF).", comment['title']):
        continue
    
    if match := re.match("Jason Benn (replied to|commented on) ([\w \-']+)'s (photo|post|comment|video|GIF)\.", comment['title']):
        name = unidecode.unidecode(match.group(2))
    
    if not name:
        print(comment)
        raise Exception(f"case not handled: {comment['title']}")
    
    interactions[name].add(comment_time.date())

## Compress info for Notion

In [14]:
compressed_interactions = {}
for name, touches in interactions.items():
    compressed_interactions[name] = {'last_interaction': max(touches), 'num_interaction_days': len(touches)}

print(sorted(compressed_interactions.items(), key=lambda x: -x[1]['num_interaction_days'])[:3])
print(max(compressed_interactions.values(), key=lambda x: x['last_interaction']))

[('Rose Wang', {'last_interaction': datetime.date(2022, 2, 16), 'num_interaction_days': 334}), ('Arram Sabeti', {'last_interaction': datetime.date(2022, 2, 13), 'num_interaction_days': 224}), ('Daniel Benn', {'last_interaction': datetime.date(2022, 2, 13), 'num_interaction_days': 159})]
{'last_interaction': datetime.date(2022, 2, 16), 'num_interaction_days': 118}


## Download from Notion

In [15]:
import requests

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()
NOTION_SECRET = os.getenv('NOTION_SECRET')
DB_ID = os.getenv('DATABASE_ID')

In [17]:
headers = {'Authorization': f'Bearer {NOTION_SECRET}', 'Notion-Version': '2021-08-16', 'Content-Type': 'application/json'}

In [18]:
from typing import Optional

def list_db(cursor: Optional[str]):
    data = {
      "sorts": [
        {
          "property": "Name",
          "direction": "ascending"
        }
      ],
      "page_size": 100
    }
    if cursor:
        data['start_cursor'] = cursor
    response = requests.post(f"https://api.notion.com/v1/databases/{DB_ID}/query", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    return json.loads(response.text)

In [19]:
pages = list_db(None)
all_pages = pages['results']

while pages['has_more']:
    print(pages['next_cursor'])
    cursor = pages['next_cursor'] # '8fefddc1-0cb6-4f66-bb07-448073cde34b',
    pages = list_db(cursor)
    all_pages.extend(pages['results'])

print(len(all_pages))

ade4fef8-9154-4ea1-a831-9fa40e167a60
ce5da540-45b5-4200-bb0c-487e28ef254f
7aa3602e-8409-4b05-8785-be92d68e1acb
bcbf67d8-70a6-41ac-9e99-a8cc4b5a46cf
469


In [27]:
all_pages[-2:]

[{'object': 'page',
  'id': 'b7dd1d92-1a20-4bcb-835f-c4dbd401af4c',
  'created_time': '2018-12-14T02:51:00.000Z',
  'last_edited_time': '2021-06-07T23:48:00.000Z',
  'cover': None,
  'icon': None,
  'parent': {'type': 'database_id',
   'database_id': '2f615374-71d6-4420-b40c-263ea48ba9e8'},
  'archived': False,
  'properties': {'Location': {'id': '%24Y%5E%26',
    'type': 'multi_select',
    'multi_select': [{'id': '818b8ab2-3014-4f92-948e-ce49ccabe3f4',
      'name': 'Bay Area',
      'color': 'red'}]},
   '❤️': {'id': '(d_(', 'type': 'checkbox', 'checkbox': False},
   'Roles': {'id': '-aRY',
    'type': 'multi_select',
    'multi_select': [{'id': 'r"=o', 'name': 'CTO', 'color': 'green'}]},
   'Fun?': {'id': '-sRT', 'type': 'checkbox', 'checkbox': False},
   'Organizations': {'id': '.-88', 'type': 'multi_select', 'multi_select': []},
   'Next question to ask them': {'id': '3zxv',
    'type': 'rich_text',
    'rich_text': []},
   'On deck?': {'id': '7%3D%7BP', 'type': 'checkbox', 'chec

In [29]:
people_in_notion = {x['properties']['Name']['title'][0]['text']['content']: x for x in all_pages if len(x['properties']['Name']['title'])}

In [30]:
print(len(people_in_notion))

466


## Reconcile people with different names

In [31]:
names_in_notion = set(people_in_notion.keys())
names_interacted = set(compressed_interactions.keys())
print(len(names_in_notion), len(names_interacted))
print(len(names_in_notion - names_interacted), len(names_interacted - names_in_notion))

466 511
253 298


In [ ]:
# fb
#  "Jimmy 'Swift' Chen",
#  'Patricia "Patty" Mou',

#  'Jeremy Omni Genesis Nixon',
#  'Matthew P. McAteer',
#  'Mingzhu Hešeri',
#  'Norian Caporale-Berkowitz',


In [ ]:
# notion
#  "Ian's friend Diana & husband Jamie",
#  'Ricky (Chef)',
#  "Tyler (F), Laura's roomie that wrote Buttigieg speeches",
#  'Wes & Marie',


## Write pages for Logseq

In [32]:
crm_file = "- [[" + "]]\n- [[".join(sorted(names_in_notion)) + "]]"

In [33]:
open("/Users/jasonbenn/notes/pages/CRM.md", "w").write(crm_file)

9149

Click Notes -> "Re-index"

## Send to Notion

In [34]:
def get_table_metadata():
    response = requests.get(f"https://api.notion.com/v1/databases/{DB_ID}", headers=headers)
    return json.loads(response.text)
# sorted(table_metadata['properties'].keys())

In [35]:
def date_iso_format(date):
    return datetime.combine(date, datetime.min.time()).isoformat()

In [36]:
def patch_page_last_connected(page_id, dt):
    data = {'properties': 
        {
         'Last connected': {'date': {'start': date_iso_format(dt), 'end': None}},
        }
       }
    response = requests.patch(f"https://api.notion.com/v1/pages/{page_id}", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    return response

In [37]:
def patch_page_days_interacted_lately(page_id, days_interacted_lately):
    data = {'properties': 
        {
          'Days interacted lately': days_interacted_lately
        }
       }
    response = requests.patch(f"https://api.notion.com/v1/pages/{page_id}", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    return response

In [ ]:
# for name, interactions in compressed_interactions.items():
#     if name in results_dict:
#         print(f"found {name}")
#     else:
#         print(f"nope  {name}")

In [38]:
from tqdm import tqdm

interacted_but_not_in_notion = []
for name, touches in tqdm(compressed_interactions.items()):
    if name in people_in_notion:
        page_id = people_in_notion[name]['id']
        patch_page_last_connected(page_id, touches['last_interaction'])
        patch_page_days_interacted_lately(page_id, touches['num_interaction_days'])
    else:
        interacted_but_not_in_notion.append(name)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 511/511 [03:13<00:00,  2.64it/s]


## Update Acquaintances Page

In [39]:
def notion_retrieve_block_children(block_id, cursor = None):
    url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    if cursor is not None:
        url += f"?start_cursor={cursor}"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return json.loads(response.text)

In [40]:
acquaintances_id = "d1ac5a638f784ca1ac96fc9509cf962c"
page_blocks = notion_retrieve_block_children(acquaintances_id)

all_blocks = page_blocks['results']
while page_blocks['has_more']:
    print(page_blocks['next_cursor'])
    cursor = page_blocks['next_cursor'] # '8fefddc1-0cb6-4f66-bb07-448073cde34b',
    page_blocks = notion_retrieve_block_children(acquaintances_id, cursor)
    all_blocks.extend(page_blocks['results'])

708a460f-198c-418b-967e-26434cab1f32
503707df-2148-4de4-b9c8-10d4b3e51061


In [42]:
existing_acqs = set(x['paragraph']['text'][0]['text']['content'] for x in all_blocks)
new_acqs = sorted(set(interacted_but_not_in_notion) - existing_acqs)
print("Should any of these recently interacted folks become FRIENDS?!")
print(new_acqs)

Should any of these recently interacted folks become FRIENDS?!
['Adi Melamed', 'Akshaya Dinesh', 'Alexander Green', 'Allegra Thaler', 'Allison Dman', 'Alok Singh', 'Angela Kong', 'Angie Palm', 'Anna Abanes', 'Anna Rulloda', 'Ari Kagan', 'Aston Motes', 'Ben Saccone', 'Bernice Yu', 'Bharat Batra', 'Bora Yu', 'Charlie Stigler', 'Chiara Cecchini', 'Chuck Terpo', 'Dani Suntoucher', 'Daniel Ricciardelli', 'Danielle Cleaver', 'Danny Boy', 'Elin Ahlstrand', 'Emma Blumke', 'Eric Ouyang', 'Ethan Caballero', 'Everett Smith', 'Ezra Malafaia', 'Gabriela Moisescu', 'George Halsted', 'Gerta Ma', 'Grace Cheng', 'Hannah Skurnik', 'Harper Carroll', 'Irina Rish', 'Janine Leger', 'Jason Seibel', 'Jay Standish', 'Jean-Loick Michaux', 'Jeffrey Lin', 'Jiani Chen', 'John Brunsfeld', 'Jonathan Gray', 'Joseph Viviano', 'Josie Lehrer', 'Julie Tereba', 'June Lin-Arlow', 'K.C. Chowdhury', 'Kat Q Hu', 'Kiru Mehari', 'Kush Sharma', 'Kwiri Yang', 'Laura Small', 'Lauren Hope Frieband', 'Liane Decary-Chen', 'Lisa', 'Lo

In [43]:
def append_block_children(children):
    data = {'children': [
		{
			"object": "block",
			"type": "paragraph",
			"paragraph": {
				"text": [
					{
						"type": "text",
						"text": {
							"content": x,
						}
					}
				]
			}
		}
	 for x in children]
    }
    response = requests.patch(f"https://api.notion.com/v1/blocks/{acquaintances_id}/children", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    return response

In [44]:
append_block_children(new_acqs)

<Response [200]>

In [46]:
import sqlite3
con = sqlite3.connect('crm.sqlite3')

In [ ]:
# Create the table
con.execute("create table acqaintances(name, first_appeared)")

# Fill the table
con.executemany("insert into lang(name, first_appeared) values (?, ?)", langs)

# Print the table contents
for row in con.execute("select name, first_appeared from lang"):
    print(row)

print("I just deleted", con.execute("delete from lang").rowcount, "rows")

# close is not a shortcut method and it's not called automatically,
# so the connection object should be closed manually
con.close()
